In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 52, in main
  File "/usr/local/lib/python3.10/dist-packages/

In [2]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

dataset = load_dataset("mrtoy/mobile-ui-design")

dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.30k [00:00<?, ?B/s]

(…)-00000-of-00003-b9a78b2072fee3aa.parquet:   0%|          | 0.00/392M [00:00<?, ?B/s]

(…)-00001-of-00003-d1f3cb3379d4941c.parquet:   0%|          | 0.00/387M [00:00<?, ?B/s]

(…)-00002-of-00003-ae5f3fe59ef58192.parquet:   0%|          | 0.00/382M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7846 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['width', 'height', 'image', 'objects'],
        num_rows: 7846
    })
})

In [5]:
!pip install diffusers transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 25.6 MB/s eta 0:00:00


In [8]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
# Import necessary libraries
from datasets import load_dataset
from diffusers import StableDiffusionPipeline
import torch
import os

# Load the dataset
dataset = load_dataset("mrtoy/mobile-ui-design")  # Adjust this line based on your dataset source

# Assuming 'dataset' is already loaded with the 'train' data
train_data = dataset['train']

# Check the structure of the train_data
print(f"Structure of train_data: {train_data}")

# Function to generate prompts from objects
def generate_prompt_from_objects(objects):
    prompt = "This mobile UI design contains: "
    prompt += ", ".join(objects)  # Join all objects into a sentence
    return prompt

# Limit to first N prompts for testing
N = 10  # Change this to the desired number of prompts for faster testing

# Generate prompts from the 'objects' field in train_data
limited_prompts = []
for i in range(min(N, len(train_data))):  # Limit to N samples
    sample = train_data[i]
    if 'objects' in sample:
        limited_prompts.append(generate_prompt_from_objects(sample['objects']))
    else:
        print(f"No 'objects' field found in sample {i}")

# Print the first N prompts to verify
for i, prompt in enumerate(limited_prompts):
    print(f"Prompt {i+1}: {prompt}")

# Load the Stable Diffusion model
model_id = "CompVis/stable-diffusion-v1-4"  # Adjust the model ID based on your preferences
pipe = StableDiffusionPipeline.from_pretrained(model_id).to("cpu")

# Define your custom path to save images
custom_image_directory = r"C:\Users\hp\Desktop\CodeWorks"

# Ensure the directory exists, if not, create it
if not os.path.exists(custom_image_directory):
    os.makedirs(custom_image_directory)

# Function to generate an image based on the text prompt
def generate_image(prompt):
    """Generate an image using Stable Diffusion based on the prompt."""
    with torch.no_grad():
        image = pipe(prompt).images[0]  # Generate the image

    # Save the image to the custom directory
    image_filename = f"generated_image_{hash(prompt)}.png"
    image_path = os.path.join(custom_image_directory, image_filename)
    image.save(image_path)

    return image_path  # Return the file path of the saved image

# Generate images for all the limited prompts
batch_size = 2  # Adjust according to your memory
generated_images = []  # To store image file paths

for i in range(0, len(limited_prompts), batch_size):
    batch_prompts = limited_prompts[i:i + batch_size]
    try:
        with torch.no_grad():
            images = pipe(batch_prompts).images  # Generate images for the batch

        for prompt, image in zip(batch_prompts, images):
            image_path = f"generated_image_{hash(prompt)}.png"
            image_path = os.path.join(custom_image_directory, image_path)  # Save in custom directory
            image.save(image_path)  # Save the image
            generated_images.append((prompt, image_path))  # Store the prompt and image path

    except Exception as e:
        print(f"Error generating images for prompts {batch_prompts}: {e}")

# Output the first few generated image paths
for i, (prompt, image_path) in enumerate(generated_images[:5]):  # Show only the first 5 for brevity
    print(f"Prompt {i + 1}: {prompt}")
    print(f"Image saved at {image_path}\n")

# Print all files in the custom directory
print("\nFiles in the custom directory:")
files = os.listdir(custom_image_directory)
for file in files:
    print(file)


Structure of train_data: Dataset({
    features: ['width', 'height', 'image', 'objects'],
    num_rows: 7846
})
Prompt 1: This mobile UI design contains: bbox, category, color, radius, text
Prompt 2: This mobile UI design contains: bbox, category, color, radius, text
Prompt 3: This mobile UI design contains: bbox, category, color, radius, text
Prompt 4: This mobile UI design contains: bbox, category, color, radius, text
Prompt 5: This mobile UI design contains: bbox, category, color, radius, text
Prompt 6: This mobile UI design contains: bbox, category, color, radius, text
Prompt 7: This mobile UI design contains: bbox, category, color, radius, text
Prompt 8: This mobile UI design contains: bbox, category, color, radius, text
Prompt 9: This mobile UI design contains: bbox, category, color, radius, text
Prompt 10: This mobile UI design contains: bbox, category, color, radius, text


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]